<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/%E8%A8%AD%E5%82%99%E6%83%85%E5%A0%B1%E5%BC%95%E7%B6%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) インストール & インポート（既に入っていれば pip はスキップされます）
try:
    import openstudio as osd
except ModuleNotFoundError:
    # Colab では先頭に '!' が必要
    !pip -q install openstudio
    import openstudio as osd

from google.colab import files

# ---- ユーティリティ ----
def load_model(path: str) -> osd.model.Model:
    """OSM を読み込んで Model を返す。失敗時は RuntimeError。"""
    opt_m = osd.model.Model.load(osd.path(path))
    if opt_m.is_initialized():
        return opt_m.get()
    raise RuntimeError(f"OpenStudio Model の読み込みに失敗: {path}")

def pick_uploaded_osm(upload_result: dict, label: str) -> str:
    """アップロード結果から最初の .osm を選ぶ。"""
    assert upload_result, f"{label}: ファイルがアップロードされていません。"
    cands = [n for n in upload_result.keys() if n.lower().endswith(".osm")]
    assert cands, f"{label}: 拡張子 .osm のファイルが見つかりません。"
    return cands[0]  # ラベルに依らず、最初の .osm を採用

# 1) 供与側（HVACあり）をアップロード
print("引き継ぐモデルデータを選択してください（HVACあり）")
uploaded_src = files.upload()
src = pick_uploaded_osm(uploaded_src, "Source")
print("Source OSM :", src)

# 2) 受け側（gbXML取込後）をアップロード
print("システムを付加するモデルを選択してください（gbXML取込後）")
uploaded_dst = files.upload()
dst = pick_uploaded_osm(uploaded_dst, "Target")
print("Target OSM :", dst)

# 3) モデル読み込み
src_m = load_model(src)
dst_m = load_model(dst)


# 4) VRF 親機を clone & マップ化（重複 clone を避ける）
vrf_map = {}  # key: src_handle(str) -> value: dst_vrf
for vrf_src in src_m.getAirConditionerVariableRefrigerantFlows():
    opt_vrf_dst_obj = vrf_src.clone(dst_m).to_AirConditionerVariableRefrigerantFlow()
    if not opt_vrf_dst_obj.is_initialized():
        print("[Warn] VRF の型変換に失敗:", vrf_src.nameString())
        continue
    vrf_dst = opt_vrf_dst_obj.get()
    vrf_map[str(vrf_src.handle())] = vrf_dst
    print("[Clone] VRF (outdoor):", vrf_src.nameString(), "->", vrf_dst.nameString())

# 5) 端末は “親 VRF から辿って” clone & 接続（端末→親の逆参照を使わない）
processed = 0
for vrf_src in src_m.getAirConditionerVariableRefrigerantFlows():
    parent_dst = vrf_map.get(str(vrf_src.handle()))
    if parent_dst is None:
        continue
    try:
        for term_src in vrf_src.terminals():
            opt = term_src.clone(dst_m).to_ZoneHVACTerminalUnitVariableRefrigerantFlow()
            if opt.is_initialized() and parent_dst.addTerminal(opt.get()):
                processed += 1
    except Exception:
        pass

print(f"[Info] VRF端末をクローンして親に接続（ゾーン未割当のまま）: {processed} 台")

# 6) その他ゾーン機器（複製のみ）
for fc in src_m.getZoneHVACFourPipeFanCoils():
    _ = fc.clone(dst_m)
    print("[Clone] FanCoil:", fc.nameString())

for ptac in src_m.getZoneHVACPackagedTerminalAirConditioners():
    _ = ptac.clone(dst_m)
    print("[Clone] PTAC:", ptac.nameString())

for pthp in src_m.getZoneHVACPackagedTerminalHeatPumps():
    _ = pthp.clone(dst_m)
    print("[Clone] PTHP:", pthp.nameString())

# 8) 保存 & ダウンロード
OUT = "target_with_HVAC.osm"
dst_m.save(osd.path(OUT), True)
print("Saved:", OUT)

files.download(OUT)
# ======================================================================


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 MB 3.0 MB/s eta 0:00:00
引き継ぐモデルデータを選択してください（HVACあり）


Saving 下妻_空調.osm to 下妻_空調.osm
Source OSM : 下妻_空調.osm
システムを付加するモデルを選択してください（gbXML取込後）


Saving 90度回転.osm to 90度回転.osm
Target OSM : 90度回転.osm
[Clone] VRF (outdoor): 〇EHP-2-2 -> 〇EHP-2-2
[Clone] VRF (outdoor): 〇EHP-2-1 -> 〇EHP-2-1
[Clone] VRF (outdoor): 〇EHP-1-4-1 -> 〇EHP-1-4-1
[Clone] VRF (outdoor): 〇EHP-1-3-W -> 〇EHP-1-3-W
[Clone] VRF (outdoor): 〇EHP-1-4-2 -> 〇EHP-1-4-2
[Clone] VRF (outdoor): 〇EHP-1-3-M -> 〇EHP-1-3-M
[Clone] VRF (outdoor): EHP-2-3 -> EHP-2-3
[Clone] VRF (outdoor): 〇EHP-1-2 -> 〇EHP-1-2
[Clone] VRF (outdoor): EHP-1-1 -> EHP-1-1
[Info] VRF端末をクローンして親に接続（ゾーン未割当のまま）: 11 台
Saved: target_with_HVAC.osm


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>